In [1]:
import sys
import os

# Add path to src/ directory
src_path = os.path.abspath("(...)/RefineAI/src")
if src_path not in sys.path:
    sys.path.append(src_path)

from run import main

# Finds thresholds for IH-confidence and IH-uncertainty 
# Uses only training data split into train and validation with 5 repetitions

main(
    data_path="(...)/RefineAI/experiments/hsl",
    train_name="hsl",
    measure="ih",  # or "if"
    metric="macro-f1",
    steps=[#"build", 
        #"results", #"reports", #"plot", 
        "grid", "heuristic", #"brute", "annealing" 
    ],  # whatever steps you want
    show_plots=False,
    save_plots=False  # or True, depending on what you want
)

Grid searching thresholds...

Heuristic searching thresholds...

Confidence-based rejection
Filtering threshold(s): 0.69375
Rejecting threshold(s): 0.88
Associated minimum cost: 1.370655

Uncertainty-based rejection
Filtering threshold(s): 0.95
Rejecting threshold(s): 0.76
Associated minimum cost: 1.426234


In [2]:
# Finds thresholds for IF-confidence and IF-uncertainty 
# Uses only training data split into train and validation with 5 repetitions

main(
    data_path="(...)/RefineAI/experiments/hsl",
    train_name="hsl",
    measure="if",  
    metric="macro-f1",
    steps=[#"build", you only need to call build once 
           #"results", "reports", #"plot", 
          "grid", "heuristic", #"brute", "annealing"
    ],  
    show_plots=False,
    save_plots=False  
)

Grid searching thresholds...

Heuristic searching thresholds...

Confidence-based rejection
Filtering threshold(s): 0.55
Rejecting threshold(s): 0.58
Associated minimum cost: 1.564757

Uncertainty-based rejection
Filtering threshold(s): 0.95
Rejecting threshold(s): 0.66
Associated minimum cost: 1.480380


In [1]:
from thresholds_tester import ThresholdsTester

import sys
import os

# Test the thresholds found in the test set

# Add path to src/ directory
src_path = os.path.abspath("(...)/RefineAI/src")
if src_path not in sys.path:
    sys.path.append(src_path)

tester = ThresholdsTester(
    data_path="(...)/RefineAI/experiments/hsl",
    train_name="hsl",
    test_name="hbp",
    metric="macro-f1",
    ih_uncertainty=[0.9500, 0.76], #filtering, rejecting
    ih_confidence=[0.69375, 0.88],
    if_uncertainty=[0.95000, 0.66],
    if_confidence=[0.5500, 0.58]
)

tester.final_measures()
final, macro = tester.final_results()

In [3]:
# Reorganizes the results 

final = final.reindex([1, 0, 4, 3, 2, 7, 6, 5, 10, 9, 8, 11,12]) 
final

,measure,metric,T_value,thresholds,precision_class_0,precision_class_1,recall_class_0,recall_class_1,rate_accep_0,rate_accep_1,mean_conf_uncert,f1_0,f1_1,f1-macro,acp_a
1,ih,C,0.306,0.88,0.804,0.613,0.765,0.667,0.864,0.851,0.971,0.784015,0.638861,0.711438,0.8575
0,ih,C,0.306,0.00,0.800,0.600,0.746,0.672,1.000,1.000,0.941,0.772057,0.633962,0.703010,1.0000
4,ih,U,0.000,0.76,1.000,1.000,1.000,1.000,0.220,0.179,0.858,1.000000,1.000000,1.000000,0.1995
3,ih,U,0.050,0.76,0.865,0.789,0.889,0.750,0.305,0.299,0.872,0.876836,0.769006,0.822921,0.3020
2,ih,U,0.050,0.00,0.768,0.562,0.729,0.612,1.000,1.000,0.672,0.747992,0.585935,0.666964,1.0000
7,if,C,0.000,0.58,0.775,0.594,0.752,0.623,0.890,0.910,0.732,0.763327,0.608154,0.685741,0.9000
6,if,C,0.450,0.58,0.741,0.705,0.885,0.470,0.958,0.985,0.807,0.806624,0.564000,0.685312,0.9715
5,if,C,0.450,0.00,0.746,0.681,0.873,0.478,1.000,1.000,0.798,0.804519,0.561722,0.683121,1.0000
10,if,U,0.000,0.66,0.854,0.833,0.911,0.741,0.381,0.403,0.790,0.881580,0.784311,0.832945,0.3920
9,if,U,0.050,0.66,0.902,0.833,0.920,0.800,0.424,0.373,0.812,0.910911,0.816167,0.863539,0.3985


In [4]:
# Calculates the cost for each solution, used to statistical comparison

macro = macro[['measure', 'metric', 'f1-macro','acp_a','mean_conf_uncert']]
macro = round(macro,3)

macro['cost'] = 4*(1-macro['f1-macro']) + (1-(macro['acp_a'])) + (1-macro['mean_conf_uncert'])

macro_confidence = macro.reindex([1, 6, 11])
macro_uncertainty = macro.reindex([3, 9, 12])

print(','.join([f"'{x:.3}'" for x in macro_uncertainty.iloc[:,-1]]))

'1.53','1.33','1.61'


In [5]:
print(','.join([f"'{x:.3}'" for x in macro_confidence.iloc[:,-1]]))

'1.33','1.48','1.55'
